In [1]:
import os
import shutil
from pathlib import Path
from datetime import datetime

import joblib
import matplotlib.pyplot as plt

from portfawn.market_data import MarketData
from portfawn.portfolio import MultiPortoflio
from portfawn.portfolio import BackTest

In [2]:
## Parameters

# market
tickers = {"S&P 500": "IVV", 
           "Small-Cap": "IJR",
           'ACWI ex U.S.': 'ACWX',
           "Emerging Markets": "IEMG", 
           "Global REIT": "REET", 
           'U.S. Real Estate': 'IYR',
           'Corporate Bond ': 'HYG',
           'Aggregate Bond': 'AGG',
           'Gold': 'GLD',
          }

start_date = datetime.strptime("2015-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2020-12-31", "%Y-%m-%d").date()
training_days = 8 * 5
testing_days = 5
col_price = 'Close'

# portfolio
portfolio_fitness = {"MSRP": "Max Sharpe-Ratio", 
                     "EWP": "Equal Weight", 
                     'MRP': "Maximum Returns",
                     'MVP': 'Minimum Volatility',
                     'SA': 'Simulated Annealing'}

portfolio_fitness_list = list(portfolio_fitness.keys())
risk_free_rate = 0.00
sampling_params = {"type": "standard"}
optimization_params = {
    "scipy_params": {
        "maxiter": 1000,
        "disp": False,
        "ftol": 1e-10,
    },
    "target_return": 0.2,
    "target_risk": 0.05,
    "weight_bound": (0.02, 0.98),
}

# system
n_jobs = joblib.cpu_count() - 1

# backtest
backtesting_name = 'test'

# Backtest config
backtesting_config = {
    "backtesting_name": backtesting_name,
    "portfolio_fitness": portfolio_fitness,
    "tickers": tickers,
    "start_date": start_date,
    "end_date": end_date,
    "optimization_params": optimization_params,
    "sampling_params": sampling_params,
    "training_days": training_days,
    "testing_days": testing_days,
    "risk_free_rate": risk_free_rate,
    "n_jobs": n_jobs,
}

# misc
file_format = 'png'

# results
path_result = Path('./plots')
shutil.rmtree(path_result)
os.makedirs(path_result)

### Market Data

In [3]:
## Create the market instance
market_data = MarketData(
    tickers = tickers,
    date_start = start_date,
    date_end = end_date,
    col_price = col_price,
)

## Plot the Cum Returns Data
fig, ax = market_data.plot_cum_returns()
fig.savefig(path_result / Path(f'market_cum_returns.{file_format}'))
plt.close()

## Dist of Returns
fig, ax = market_data.plot_dist_returns()
fig.savefig(path_result / Path(f'market_dist_returns.{file_format}'))
plt.close()

# Corr Data
fig, ax = market_data.plot_corr()
fig.savefig(path_result / Path(f'market_corr.{file_format}'))
plt.close()

[*********************100%***********************]  9 of 9 completed


/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


### Multiple Portfolio

In [4]:
data_returns = market_data.data_returns
multi_portfolio = MultiPortoflio(portfolio_fitness_list=portfolio_fitness_list,
                                 data_returns=data_returns,
                                 risk_free_rate=risk_free_rate,
                                 sampling_params=sampling_params,
                                 optimization_params=optimization_params,
                                )
multi_portfolio.generate()
fig, ax = multi_portfolio.plot_portfolio()
fig.savefig(path_result / Path(f'portfolio_multi.{file_format}'))
plt.close()

/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


### Backtesting

In [5]:
portfolio_backtest = BackTest(**backtesting_config)
portfolio_backtest.run()

## Plot the Cum Returns
fig, ax = portfolio_backtest.plot_cum_returns()
fig.savefig(path_result / Path(f'portfolio_cum_returns.{file_format}'))
plt.close()

## Dist of Returns
fig, ax = portfolio_backtest.plot_dist_returns()
fig.savefig(path_result / Path(f'portfolio_dist_returns.{file_format}'))
plt.close()

# Corr Data
fig, ax = portfolio_backtest.plot_corr()
fig.savefig(path_result / Path(f'portoflio_corr.{file_format}'))
plt.close()

[*********************100%***********************]  9 of 9 completed


/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/media/veracrypt4/projects/PortFawn/venv/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were